In [37]:
from pandas import read_excel
from pandas import DataFrame
from pandas import concat
from pandas import Series
import numpy
from numpy import append
from numpy import array
from numpy import diff
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings
warnings.filterwarnings("ignore")

In [38]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return array(diff)

In [39]:
# scale train and test data to [-1, 1]
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled

In [40]:
# Get technical indicators
def get_technical_indicators(data):
    # Create Moving Average
    data = DataFrame(data)
    dataset = DataFrame()
    dataset['ma5'] = data.rolling(window=5).mean()
    dataset['ma20'] = data.rolling(window=25).mean()
    
    # Create MACD
    dataset['25ema'] = data.ewm(span=25).mean()
    dataset['10ema'] = data.ewm(span=10).mean()
    dataset['MACD'] = dataset['10ema']-dataset['25ema']

    # Create Bollinger Bands
    dataset['20sd'] = data.rolling(window = 20).std()
    dataset['upper_band'] = dataset['ma20'] + (dataset['20sd']*2)
    dataset['lower_band'] = dataset['ma20'] - (dataset['20sd']*2)
    
    # Create Exponential moving average
    dataset['ema'] = data.ewm(com=0.5).mean()
    
    # Create Momentum
    dataset['momentum'] = data-1
    # dataset['log_momentum'] = log(dataset['momentum'])
    return dataset.dropna()

In [41]:
def timeseries_to_supervised(data, lag):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    return DataFrame(df)

In [42]:
# fit an LSTM network to training data
def model_fit(train, n_lag, n_batch, nb_epoch, n_neurons):
    # reshape training into [samples, timesteps, features]
    X, y = train[:, 0:n_lag], train[:, n_lag:]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    # design network
    model = Sequential()
    model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    # fit network
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
        model.reset_states()
    return model

In [43]:
# make a one-step forecast
def model_predict(model, X, n_batch):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=n_batch)
    return yhat[0]

In [44]:
# inverse scaling for a forecasted value
def invert_scale(scaler, X):
    inverted = scaler.inverse_transform(array(X).reshape(-1,1))
    return array(inverted)

In [45]:
# invert differenced value
def invert_difference(last_ob, yhat_diff, interval=1):
    return numpy.r_[last_ob, yhat_diff].cumsum()

In [46]:
# run a repeated experiment
def experiment(series, features):
    # transform data to be stationary
    raw_values = series.values
    diff_values = difference(raw_values)
    
    # split data into train and test-sets
    len_test = 120
    train = diff_values[:-len_test]
    test = diff_values[-len_test:]
    
    # transform the scale of the data
    scaler, train_scaled, test_scaled = scale(train, test)
    
    # get technical indicators
    df = get_technical_indicators(train_scaled)
    
    # transform data to be supervised learning
    supervised = timeseries_to_supervised(train_scaled, features)
    supervised_values = supervised.values[features:,:]
    
    data = concat([df, DataFrame(supervised_values)],axis=1).dropna()
    
    # fit the base model
    n_features = data.shape[1]-1
    n_batch = 1
    nb_epoch = 50
    n_neurons = 1
    mod = model_fit(array(data), n_features, n_batch, nb_epoch, n_neurons)

    # predict first row
    testX, testy = data.iloc[-1, :-1], data.iloc[-1, -1]

    # forecast test dataset
    predictions = list()
    for i in range(len(test)):
        # predict
        yhat_scaled = model_predict(mod, array(testX), n_batch)
        # store forecast
        train_scaled = append(train_scaled,yhat_scaled)
        df = get_technical_indicators(train_scaled)
        supervised = timeseries_to_supervised(train_scaled, features)
        supervised_values = supervised.values[features:,:]
        data = concat([df, DataFrame(supervised_values)],axis=1)
        data = DataFrame(data).dropna()
        testX = data.iloc[-1,:-1]
        predictions.append(yhat_scaled)
        
    predictions = invert_scale(scaler,predictions)
    predictions = invert_difference(raw_values[-len_test],predictions[:,0])
    
    return DataFrame(predictions)

In [47]:
path = "C:\\Users\\fh20175\\Box\\Habbab Fatima\\Experiment set 3\\DataSets\\"
asset_list = read_excel("C:\\Users\\fh20175\\Box\\Habbab Fatima\\Experiment set 6\\asset_list.xlsx")

no_assets = len(asset_list)

predictions = DataFrame()

for i in range(0,no_assets):
    ticker = asset_list.iloc[i,0]
    data = read_excel(path+ticker+".xlsx")
    series = data[["Close"]].dropna()
    
    no_features = 2
    
    yhat = experiment(series, no_features)

    predictions[ticker] = yhat

1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 19ms/step


486/486 [==============================] - 1s 1ms/step - loss: 0.0352


485/485 [==============================] - 1s 1ms/step - loss: 0.0153


...

483/483 [==============================] - 1s 1ms/step - loss: 0.0432


483/483 [==============================] - 1s 2ms/step - loss: 0.0266


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 21ms/step


In [48]:
predictions.to_excel("pred120.xlsx")